In [1]:
from google.cloud import bigquery
import bq_helper 
import numpy as np
import pandas as pd

In [2]:
# create a helper object for this dataset
accidents = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="nhtsa_traffic_fatalities")

Using Kaggle's public dataset BigQuery integration.


In [3]:
list(accidents.head('accident_2016').columns)

['state_number',
 'state_name',
 'consecutive_number',
 'number_of_vehicle_forms_submitted_all',
 'number_of_motor_vehicles_in_transport_mvit',
 'number_of_parked_working_vehicles',
 'number_of_forms_submitted_for_persons_not_in_motor_vehicles',
 'number_of_persons_not_in_motor_vehicles_in_transport_mvit',
 'number_of_persons_in_motor_vehicles_in_transport_mvit',
 'number_of_forms_submitted_for_persons_in_motor_vehicles',
 'county',
 'city',
 'day_of_crash',
 'month_of_crash',
 'year_of_crash',
 'day_of_week',
 'hour_of_crash',
 'minute_of_crash',
 'national_highway_system',
 'land_use',
 'land_use_name',
 'functional_system',
 'functional_system_name',
 'ownership',
 'ownership_name',
 'route_signing',
 'route_signing_name',
 'trafficway_identifier',
 'trafficway_identifier_2',
 'milepoint',
 'latitude',
 'longitude',
 'special_jurisdiction',
 'special_jurisdiction_name',
 'first_harmful_event',
 'first_harmful_event_name',
 'manner_of_collision',
 'manner_of_collision_name',
 'relati

In [4]:
query = """SELECT state_name,
            COUNT(consecutive_number) AS total
            FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2016`
            GROUP BY 1
            ORDER BY COUNT(consecutive_number) DESC
        """
accidents_by_states = accidents.query_to_pandas_safe(query)

In [5]:
accidents_by_states['state_name'].unique()

array(['California', 'Texas', 'Florida', 'Georgia', 'North Carolina',
       'Pennsylvania', 'Ohio', 'Illinois', 'New York', 'Michigan',
       'Alabama', 'Tennessee', 'South Carolina', 'Missouri', 'Arizona',
       'Indiana', 'Kentucky', 'Virginia', 'Louisiana', 'Oklahoma',
       'Mississippi', 'New Jersey', 'Colorado', 'Wisconsin', 'Washington',
       'Arkansas', 'Maryland', 'Oregon', 'Kansas', 'New Mexico',
       'Minnesota', 'Massachusetts', 'Iowa', 'Nevada', 'Connecticut',
       'Utah', 'West Virginia', 'Idaho', 'Nebraska', 'Montana', 'Maine',
       'New Hampshire', 'Delaware', 'Hawaii', 'South Dakota',
       'North Dakota', 'Wyoming', 'Alaska', 'Vermont', 'Rhode Island',
       'District of Columbia'], dtype=object)

## Top 10 States by Traffic fatalities by 2016 in USA

In [6]:
accidents_by_states.head(10)

,state_name,total
0,California,3540
1,Texas,3427
2,Florida,2935
3,Georgia,1424
4,North Carolina,1348
5,Pennsylvania,1088
6,Ohio,1053
7,Illinois,1000
8,New York,981
9,Michigan,981


In [8]:
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
number = accidents_by_states.total.head(10)
name = accidents_by_states.state_name.head(10)


trace0 = go.Bar(x=number, y=name, orientation = 'h')
data = [trace0]

layout = go.Layout(title='Top 10 States Total Traffic Fatalities in 2016')

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Fatalities_2016')

In [9]:
!pip install folium

## Mapping states fatalities 

In [12]:
query = """SELECT DISTINCT(state_name),
             latitude, longitude
            FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2016`
                     
        """
location = accidents.query_to_pandas_safe(query)

In [13]:
states_cordinates = location.groupby(['state_name'])['latitude', 'longitude'].mean()

In [14]:
states_cordinates = states_cordinates.reset_index()

In [15]:
states_cordinates.columns

Index(['state_name', 'latitude', 'longitude'], dtype='object')

In [16]:
accidents_by_states.sort_values('state_name', inplace=True)

In [17]:
accidents_by_states

,state_name,total
10,Alabama,979
47,Alaska,78
14,Arizona,856
25,Arkansas,504
0,California,3540
22,Colorado,558
34,Connecticut,292
42,Delaware,116
50,District of Columbia,26
2,Florida,2935


In [18]:
sates_df = states_cordinates.merge(accidents_by_states,on='state_name', how='inner')

In [19]:
sates_df

,state_name,latitude,longitude,total
0,Alabama,33.035266,-85.597817,979
1,Alaska,61.492575,-148.719861,78
2,Arizona,33.658046,-110.549605,856
3,Arkansas,35.086740,-92.575877,504
4,California,35.760657,-118.827687,3540
5,Colorado,39.466803,-105.298593,558
6,Connecticut,41.500714,-72.824792,292
7,Delaware,39.206352,-75.542161,116
8,District of Columbia,38.888933,-76.994510,26
9,Florida,28.019679,-81.782806,2935


In [20]:
import folium
from folium import plugins
map_obj = folium.Map(location=[42.50, -99.45], tiles='cartodbpositron', zoom_start=3.5)
for j, rown in sates_df.iterrows():
    rown = list(rown)
    folium.CircleMarker([float(rown[1]), float(rown[2])], popup="<b>State:</b>" + rown[0].title() +"<br> <b>Fatalites:</b> "+str(int(rown[3])), radius=float(rown[3])*0.001, color='#be0eef', fill=True).add_to(map_obj)
map_obj